In [8]:
import requests
import pandas as pd
import boto3
from datetime import datetime as dt
from datetime import timedelta as td



In [9]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table= dynamodb.Table('jobs')
    job_details= table.get_item(Key={'job_id':job_name})['Item']
    return job_details
      

In [10]:
job_details = get_job_details('ghactivity_ingest')
job_details

{'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3'),
 'job_run_bookmark_details': {'last_run_file_name': '2023-08-10-0.json.gz'}}

In [11]:
baseline_days = job_details['baseline_days']
baseline_days

Decimal('3')

In [12]:
start_file = f'{dt.strftime(dt.now().date()-td(days=3),"%Y-%m-%d")}-0.json.gz'
start_file

'2023-08-13-0.json.gz'

In [13]:
job_details = {'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': 3,
 "job_run_bookmark_details":{
     'last_run_file_name': start_file,
 }
}

In [14]:
dynamodb=boto3.resource('dynamodb')
table = dynamodb.Table('jobs')


In [15]:
table.put_item(Item=job_details)


{'ResponseMetadata': {'RequestId': 'D2G1QB5MEB9UOVT4NPPSVR1UJJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 16 Aug 2023 14:20:51 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'D2G1QB5MEB9UOVT4NPPSVR1UJJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [16]:
table.get_item(Key={'job_id':'ghactivity_ingest'})

{'Item': {'job_description': 'Ingest ghactivity data to s3',
  'is_active': 'Y',
  'job_id': 'ghactivity_ingest',
  'baseline_days': Decimal('3'),
  'job_run_bookmark_details': {'last_run_file_name': '2023-08-13-0.json.gz'}},
 'ResponseMetadata': {'RequestId': '5U25FT34J0N3G3P949SRI262P7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 16 Aug 2023 14:20:51 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '233',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5U25FT34J0N3G3P949SRI262P7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1541871833'},
  'RetryAttempts': 0}}

In [17]:
job_run_bookmark_details = table.\
    get_item(Key={'job_id':'ghactivity_ingest'})['Item']['job_run_bookmark_details']
job_run_bookmark_details

{'last_run_file_name': '2023-08-13-0.json.gz'}

In [18]:
dt_part = job_run_bookmark_details['last_run_file_name'].split('.')[0]
dt_part

'2023-08-13-0'

In [19]:
next_file = f"{dt.strftime(dt.strptime(dt_part,'%Y-%m-%d-%H') + td(hours=1),'%Y-%m-%d-%-H')}.json.gz"
next_file

'2023-08-13-1.json.gz'

In [20]:
res = requests.get(f'https://data.gharchive.org/{next_file}')
file = open(f'data/{next_file}', 'wb')
file.write(res.content)
file.close()

In [21]:
!ls -ltr data/

total 213624
-rw-r--r-- 1 admin123 admin123 33989636 Aug  8 17:12 2022-06-05-0.json.gz
-rw-r--r-- 1 admin123 admin123 75005991 Aug 13 17:47 2023-08-10-0.json.gz
-rw-r--r-- 1 admin123 admin123 71020062 Aug 16 14:37 2023-08-10-1.json.gz
-rw-r--r-- 1 admin123 admin123 38724119 Aug 16 19:51 2023-08-13-1.json.gz


In [22]:
df = pd.read_json(f'data/{next_file}',lines=True,orient='records')
df

,id,type,actor,repo,payload,public,created_at,org
0,31080082626,CreateEvent,"{'id': 1039839, 'login': 'santina', 'display_l...","{'id': 592891776, 'name': 'openseattle/open-se...","{'ref': 'santina/update_rebrand2', 'ref_type':...",True,2023-08-13 01:00:00+00:00,"{'id': 3466034, 'login': 'openseattle', 'grava..."
1,31080082627,CreateEvent,"{'id': 51386346, 'login': 'c3-github-integrati...","{'id': 677883738, 'name': 'c3-github-integrati...","{'ref': None, 'ref_type': 'repository', 'maste...",True,2023-08-13 01:00:00+00:00,NaN
2,31080082628,PushEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 620880430, 'name': '5Lin/ChatGPT-Next-W...","{'repository_id': 620880430, 'push_id': 146561...",True,2023-08-13 01:00:00+00:00,NaN
3,31080082629,WatchEvent,"{'id': 94348047, 'login': 'ltt159264', 'displa...","{'id': 321316812, 'name': 'Lightning-Universe/...",{'action': 'started'},True,2023-08-13 01:00:00+00:00,"{'id': 120740856, 'login': 'Lightning-Universe..."
4,31080082631,CreateEvent,"{'id': 66220547, 'login': 'Victoria240', 'disp...","{'id': 650357160, 'name': 'Victoria240/react-t...","{'ref': 'lesson_1_7', 'ref_type': 'branch', 'm...",True,2023-08-13 01:00:00+00:00,NaN
...,...,...,...,...,...,...,...,...
122992,31080413256,PushEvent,"{'id': 111329684, 'login': 'B74LABgit', 'displ...","{'id': 534383388, 'name': 'B74LABgit/CAM', 'ur...","{'repository_id': 534383388, 'push_id': 146563...",True,2023-08-13 01:59:59+00:00,NaN
122993,31080413253,PushEvent,"{'id': 37741296, 'login': 'xinonria', 'display...","{'id': 677026427, 'name': 'xinonria/xinonria.g...","{'repository_id': 677026427, 'push_id': 146563...",True,2023-08-13 01:59:59+00:00,NaN
122994,31080413259,CreateEvent,"{'id': 113607660, 'login': 'josh270', 'display...","{'id': 677892162, 'name': 'josh270/password-ge...","{'ref': 'main', 'ref_type': 'branch', 'master_...",True,2023-08-13 01:59:59+00:00,NaN
122995,31080413263,PullRequestEvent,"{'id': 13025337, 'login': 'bitnami-bot', 'disp...","{'id': 476381845, 'name': 'bitnami/containers'...","{'action': 'closed', 'number': 44472, 'pull_re...",True,2023-08-13 01:59:59+00:00,"{'id': 5446553, 'login': 'bitnami', 'gravatar_..."


In [24]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')

In [28]:
df.dtypes

id                          int64
type                       object
actor                      object
repo                       object
payload                    object
public                       bool
created_at    datetime64[ns, UTC]
org                        object
dtype: object

In [30]:
df.shape

(122997, 8)